<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

# Point depostion data to Deposition on parcel
A Point based depsoiton file as input will be used to find the deposition data on each parcel

In [ ]:
class Point():
    def __init__(self, id, x, y, z):
        self.id = id
        self.x = x
        self.y = y
        self.z = z


class Vertex():
    def __init__(self, x, y):
        self.x = x
        self.y = y


class Extension:
    def __init__(self, xmin, ymin, xmax, ymax, circular=False):
        self.xmax = xmax
        self.xmin = xmin
        self.ymax = ymax
        self.ymin = ymin
        self.dx = xmax - xmin
        self.dy = ymax - ymin
        self.circular = circular;
        self.radius = 0
        self.xc = (self.xmax + self.xmin) * 0.5
        self.yc = (self.ymax + self.ymin) * 0.5

        if circular:
            self.radius = math.sqrt(pow(self.dx, 2) + pow(self.dy, 2)) * 0.5

        self.vertex = [Vertex(xmin, ymin),
                       Vertex(xmax, ymin),
                       Vertex(xmax, ymax),
                       Vertex(xmin, ymax)]

    def contains(self, point):

        dist = 0
        if self.circular:
            dist = math.sqrt(pow(self.xc - point.x, 2) + pow(self.yc - point.y, 2))
            if dist <= self.radius:
                return True

        if (self.xmax >= point.x and self.ymax >= point.y and self.xmin < point.x and self.ymin < point.y):
            return True
        return False

    def intersect(self, extension):

        distv0 = 0
        distv1 = 0
        distv2 = 0
        distv3 = 0

        if extension.circular:
            distv0 = math.sqrt(pow(extension.xc - self.xmin, 2) + pow(extension.yc - self.ymin, 2))
            distv1 = math.sqrt(pow(extension.xc - self.xmax, 2) + pow(extension.yc - self.ymin, 2))
            distv2 = math.sqrt(pow(extension.xc - self.xmax, 2) + pow(extension.yc - self.ymax, 2))
            distv3 = math.sqrt(pow(extension.xc - self.xmin, 2) + pow(extension.yc - self.ymax, 2))

            if self.contains(Point(0, extension.xmax, extension.yc, 0)):
                return True
            if self.contains(Point(0, extension.xc, extension.ymax, 0)):
                return True
            if self.contains(Point(0, extension.xmin, extension.yc, 0)):
                return True
            if self.contains(Point(0, extension.xc, extension.ymin, 0)):
                return True
            if self.contains(Point(0, extension.xc, extension.yc, 0)):
                return True
            if distv0 <= extension.radius:
                return True
            if distv1 <= extension.radius:
                return True
            if distv2 <= extension.radius:
                return True
            if distv3 <= extension.radius:
                return True

            return False

        if not (extension.xmin > self.xmax or
                extension.xmax < self.xmin or
                extension.ymin > self.ymax or
                extension.ymax < self.ymin):
            return True


class QuadTree():

    def __init__(self, extension, capacity):
        self.extension = extension
        self.capacity = capacity
        self.points = [];
        self.divided = False

    def insertPoints(self, points):
        for point in points:
            self.insertPoint(point)

    def insertPoint(self, point):

        if not self.extension.contains(point):
            return False

        if len(self.points) < self.capacity:
            self.points.append(point)
            return True
        else:
            if not self.divided:
                self.subdivide()

            if self.NW.insertPoint(point):
                return True
            elif self.NE.insertPoint(point):
                return True
            elif self.SE.insertPoint(point):
                return True
            elif self.SW.insertPoint(point):
                return True

    def subdivide(self):
        exNE = Extension((self.extension.xmin + self.extension.xmax) * 0.5,
                         (self.extension.ymin + self.extension.ymax) * 0.5,
                         self.extension.xmax, self.extension.ymax)
        exNW = Extension(self.extension.xmin,
                         (self.extension.ymin + self.extension.ymax) * 0.5,
                         (self.extension.xmin + self.extension.xmax) * 0.5, self.extension.ymax)
        exSW = Extension(self.extension.xmin, self.extension.ymin, (self.extension.xmin + self.extension.xmax) * 0.5,
                         (self.extension.ymin + self.extension.ymax) * 0.5)
        exSE = Extension((self.extension.xmin + self.extension.xmax) * 0.5, self.extension.ymin,
                         self.extension.xmax, (self.extension.ymin + self.extension.ymax) * 0.5)
        self.NE = QuadTree(exNE, self.capacity)
        self.NW = QuadTree(exNW, self.capacity)
        self.SW = QuadTree(exSW, self.capacity)
        self.SE = QuadTree(exSE, self.capacity)
        self.divided = True

    def select(self, extension):
        found = []
        return self.query(extension, found)

    def query(self, extension, found=[]):

        # if not found:
        #   found=[]

        if not self.extension.intersect(extension):
            return
        else:
            for point in self.points:
                if extension.contains(point):
                    found.append(point)
            if self.divided:
                self.NE.query(extension, found)
                self.NW.query(extension, found)
                self.SW.query(extension, found)
                self.SE.query(extension, found)
        # print(len(found))
        return found

    # todo make circular
    def Plot(self, extension=[], points=[]):
        fig = plt.figure(figsize=(12, 8))
        plt.title("Quadtree")
        ax = fig.add_subplot(111)
        self.graph(ax)
        if points:
            x = [point.x for point in points]
            y = [point.y for point in points]
            plt.plot(x, y, 'bo')
        if extension:
            if extension.circular:
                ax.add_patch(patches.Circle((extension.xc, extension.yc), extension.radius, fill=False, color='blue'))
            else:
                ax.add_patch(patches.Rectangle((extension.xmin, extension.ymin), extension.dx, extension.dy, fill=False,
                                               color='blue'))
        plt.show()

    def graph(self, ax):

        ax.add_patch(patches.Rectangle((self.extension.xmin, self.extension.ymin), self.extension.dx, self.extension.dy,
                                       fill=False))
        # print(len(self.points))
        x = [point.x for point in self.points]
        y = [point.y for point in self.points]
        plt.plot(x, y, 'ro')

        if (self.divided):
            self.NE.graph(ax)
            self.NW.graph(ax)
            self.SW.graph(ax)
            self.SE.graph(ax)
        return


def select(extension, points):
    pointList = []

    for point in points:
        if extension.contains(point):
            pointList.append(point)
    return pointList


def get_extension(country):

    with fiona.open(country, 'r') as extension_area:
        extension_coor = extension_area.bounds
        extension = Extension(extension_coor[0], extension_coor[1], extension_coor[2], extension_coor[3])
        return extension


def prepare_points(depositon_path):
    points = []
    with fiona.open(depositon_path, 'r') as deposition_points:
        for point in deposition_points:
            fid = int(point['id'])
            # todo value Lambert_X en Lambert_Y
            geom_x = point['geometry']['coordinates'][0]
            geom_y = point['geometry']['coordinates'][1]
            value_deposition = point['properties']['value']
            dep_points = Point(fid, geom_x, geom_y, value_deposition)
            points.append(dep_points)
    return points

Inverse distance weight function to interpolate the depsotion value between points

In [ ]:
def CalcIDWvalue(unknowncell, knowncells, sensitivity):
    weighted_values_sum = 0.0
    sum_of_weights = 0.0
    neg_half_sens = -sensitivity/2.0
    for knowncell in knowncells:
        weight = ((unknowncell.x-knowncell.x)**2 + (unknowncell.y-knowncell.y)**2)**neg_half_sens
        sum_of_weights += weight
        weighted_values_sum += weight * knowncell.z
    
    return weighted_values_sum / sum_of_weights

Interpolation mechanism

In [ ]:
def interpol_deposition(res_intersection, qtree,distance_step=500, points_needed = 1, circular=True):


    polygon_deposition_list = []


    for index, polygon in res_intersection.iterrows():
        centroid = polygon['geometry'].centroid

        found = []
        distance = 0
        while len(found) < points_needed:
            #todo check if influence with far away polygon
            if distance > (float(distance_step)*10):
                break
            else:
                # extension for the query
                x0 = centroid.x - distance
                x1 = centroid.x + distance
                y0 = centroid.y - distance
                y1 = centroid.y + distance

                extQuery = Extension(min(x0, x1), min(y0, y1), max(x0, x1), max(y0, y1), circular)

                found = qtree.select(extQuery)
                distance += float(distance_step)


        if len(found) >= points_needed:
            idw_value = CalcIDWvalue(unknowncell=centroid, knowncells=found, sensitivity=3)
        else:
            idw_value = 0

        polygon_deposition_list.append(float(idw_value))

    res_intersection['deposition'] = polygon_deposition_list

    return res_intersection

In [ ]:
def AOI_creation(AOI_path, soil_path, parcel_path, deposition_path):

    config = configparser.ConfigParser()
    config.read(f'{os.getcwd()}/Assets/config.ini')
    
    
    # Data

    country = parcel_path

    crop_dictionary = os.getcwd() + '/' + config["ModelInputs"]["CropGroup"]
    crop_column = "LBLHFDTLT"
    soil_dictionary =  os.getcwd() + '/' +config["ModelInputs"]["SoilGroup"]
    soil_column = "Grove_leg"
    

    deposition_interpolation_distance = 100

    # get AOI

    aoi_mask = gpd.read_file(AOI_path)
    aoi_bbox = tuple(aoi_mask.total_bounds)


    # get crops
    complete_crop = gpd.read_file(parcel_path, encoding='utf-8', bbox=aoi_bbox)
    crop = gpd.clip(complete_crop, aoi_mask, keep_geom_type=True)
    
    with open(crop_dictionary, 'r', encoding='utf-8') as dict:
        aoi_specific_dictionary = json.load(dict)
        crop['reclass_crop'] = crop[crop_column].replace(aoi_specific_dictionary)
        AOI_crop = crop[['fid',crop_column,'reclass_crop','geometry']]
    
    # get soil
    complete_soil = gpd.read_file(soil_path, encoding='utf-8', bbox=aoi_bbox)
    soil = gpd.clip(complete_soil, aoi_mask, keep_geom_type=True)

    with open(soil_dictionary, 'r', encoding='utf-8') as dict:
        aoi_specific_dictionary = json.load(dict)
        soil['reclass_soil'] = soil[soil_column].replace(aoi_specific_dictionary)
        AOI_soil = soil[['fid',soil_column,'reclass_soil','geometry']]



    # Merge Parcel with soils
    soil_ind = AOI_soil
    soil_ind.sindex

    crop_ind = AOI_crop
    crop_ind.sindex

    # overlay much slower than sjoin
    res_intersection = gpd.sjoin(crop_ind, soil_ind, predicate='intersects')

    # dissolve to get the initial crop size parcels
    res_intersection = res_intersection.dissolve(by='fid_left', aggfunc='first', as_index=False)
    res_intersection = res_intersection[['fid_left', f'{crop_column}', 'reclass_crop',  f'{soil_column}', 'reclass_soil', 'geometry']]
    res_intersection.columns = ['id', 'crop_type', 'crop_class', 'soil_type', 'soil_class', 'geometry']

    # only real crops, exclude agr. infrastructure
    df = res_intersection.replace(to_replace='None', value=np.nan).dropna()


    # remove Multipolygon geometries
    id_to_remove = []
    for parcel in df.itertuples():
        if parcel.geometry.geom_type == 'MultiPolygon':
            id_to_remove.append(parcel.id)
        else:
            pass

    df = df.query(f'id not in {id_to_remove}')
    print(f'Parcels {id_to_remove} are removed froma AOI because of Multipolygon problems, inspect AOI borders')

    # Deposition ofeach parcel
    #country_shape = gpd.read_file(country)
    extension = get_extension(country)
    qtree = QuadTree(extension, 5000)
    points = prepare_points(deposition_path)

    qtree.insertPoints(points)

    # todo check if expanding window works
    parcel_df = interpol_deposition(df, qtree, distance_step=deposition_interpolation_distance)

    return parcel_df